# Simple classifier

In [18]:
%run -i "../util/file_utils.ipynb"
%run -i "../util/lang_utils.ipynb"

In [29]:
from datasets import load_dataset
train_dataset = load_dataset("rotten_tomatoes", split="train[:15%]+train[-15%:]")
test_dataset = load_dataset("rotten_tomatoes", split="test[:15%]+test[-15%:]")

Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


In [30]:
print(len(train_dataset))
print(len(test_dataset))

2560
320


In [31]:
class POS_vectorizer:
    def __init__(self, spacy_model):
        self.model = spacy_model
    
    def vectorize(self, input_text):
        doc = self.model(input_text)
        vector = []
        vector.append(len(doc))
        pos = {"VERB":0, "NOUN":0, "PROPN":0, "ADJ":0, "ADV":0, "AUX":0, "PRON":0, "NUM":0, "PUNCT":0}
        for token in doc:
            if token.pos_ in pos:
                pos[token.pos_] += 1
        vector_values = list(pos.values())
        vector = vector + vector_values
        return vector

In [32]:
sample_text = train_dataset[0]["text"]
vectorizer = POS_vectorizer(small_model)
vector = vectorizer.vectorize(sample_text)
print(vector)

[38, 3, 8, 3, 4, 1, 3, 1, 0, 5]


In [33]:
import pandas as pd
train_df = train_dataset.to_pandas()
train_df.sample(frac=1)
test_df = test_dataset.to_pandas()
train_df["vector"] = train_df["text"].apply(lambda x: vectorizer.vectorize(x))
test_df["vector"] = test_df["text"].apply(lambda x: vectorizer.vectorize(x))
X_train = np.stack(train_df["vector"].values, axis=0)
X_test = np.stack(test_df["vector"].values, axis=0)
y_train = train_df["label"].to_numpy()
y_test = test_df["label"].to_numpy()

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
clf = LogisticRegression(C=0.1)
clf = clf.fit(X_train, y_train)

In [41]:
test_df["prediction"] = test_df["vector"].apply(lambda x: clf.predict([x])[0])
print(classification_report(test_df["label"], test_df["prediction"]))

              precision    recall  f1-score   support

           0       0.59      0.54      0.56       160
           1       0.57      0.62      0.60       160

    accuracy                           0.58       320
   macro avg       0.58      0.58      0.58       320
weighted avg       0.58      0.58      0.58       320

